In [139]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as  nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
import time
from tqdm.autonotebook import tqdm, trange
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

companies = ['GAZP', 'SBER', 'LKOH', 'GMKN', 'MGNT',
       'TATN', 'NVTK', 'SNGS', 'PLZL', 'SNGSP',
       'PIKK', 'ROSN', 'SBERP', 'CHMF', 'NLMK',
       'IRAO', 'YNDX', 'ALRS', 'RUAL', 'MTSS',
       'MAGN', 'PHOR', 'RTKM', 'VTBR', 'TATNP',
       'TCSG', 'AGRO', 'OZON', 'AFLT', 'AFKS',
       'FEES', 'TRNFP', 'CBOM', 'VKCO', 'ENPG',
       'SGZH', 'FIVE', 'POLY', 'MOEX', 'GLTR',
       'HYDR', 'FIXP']


def hour2time(hour):
    if hour == 1:
        res = '11:00:00'
    elif hour == 2:
        res = '12:00:00'
    elif hour == 3:
        res = '13:00:00'
    elif hour == 4:
        res = '14:00:00'
    elif hour == 5:
        res = '15:00:00'
    elif hour == 6:
        res = '16:00:00'
    elif hour == 7:
        res = '17:00:00'
    elif hour == 8:
        res = '18:00:00'
    elif hour == 9:
        res = '18:45:00'
    return res

In [144]:
path = '~/Рабочий стол/ДИПЛОМ НОУТБУКИ/DL forcasitng diplom/Данные Мосбиржа/Total_MOEX_(NEW)hourly_with_time_df'
total_df = pd.read_csv(path, index_col='tradedate')

In [145]:
total_df = total_df[ ('2023-02-03'<=total_df.index) * (total_df.index<= '2023-10-30')]

In [146]:
total_df['date'] = total_df.index

res = []
for i in range(total_df.shape[0]):
    res.append(np.mean((total_df.close[i], total_df.open[i]) ) )
total_df['mean_price'] = res   
total_df['vol_in_rub'] = total_df.vol * total_df.mean_price

In [148]:
total_df

res = [1]
h = 2
for i in range(1, 68639):
    if total_df.iloc[i,3] == total_df.iloc[i-1,3]:
        res.append(h)
        h+=1
    if total_df.iloc[i,3] != total_df.iloc[i-1,3]:
        h=1
        res.append(h)
        h+=1
total_df['hour'] = res

In [149]:
volumes = total_df.groupby('date').aggregate({'vol_in_rub':'sum'})

In [150]:
volumes.to_csv('market_volumes')

# Closes

In [120]:
df = pd.read_csv('backtest_data').rename(columns={'Unnamed: 0':'date'})

In [121]:
dd = df['date'].iloc[:,1]
df = df.drop(columns='date')
df['date'] = dd

In [122]:
bad_days = df.groupby('date').aggregate({'Asset':'count'}).query("Asset < 200").index

In [123]:
for i in range(0,len(bad_days)):    
    df = df[df.date != bad_days[i]]

In [124]:
df['Date'] = pd.to_datetime(df.date + ' ' + df.hour.apply(hour2time) )
df.drop(['date','hour','pct_pred','pct_fact','Pred'],1,inplace=True)

/tmp/ipykernel_8281/3486096165.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['date','hour','pct_pred','pct_fact','Pred'],1,inplace=True)


In [125]:
pivot_df = df.pivot(index='Date', columns='Asset', values='Fact')


# Заменим названия колонок для удобства
pivot_df.columns.name = None

In [127]:
pivot_df.to_csv('cvxportfolio_fact_closes')

# Returns

In [128]:
df = pd.read_csv('backtest_data').rename(columns={'Unnamed: 0':'date'})

In [129]:
dd = df['date'].iloc[:,1]
df = df.drop(columns='date')
df['date'] = dd

In [130]:
bad_days = df.groupby('date').aggregate({'Asset':'count'}).query("Asset < 200").index

In [131]:
for i in range(0,len(bad_days)):    
    df = df[df.date != bad_days[i]]

In [134]:
df['Date'] = pd.to_datetime(df.date + ' ' + df.hour.apply(hour2time) )

df.drop(['date','hour','pct_fact','Pred','Fact'],1,inplace=True)

/tmp/ipykernel_8281/2968876676.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['date','hour','pct_fact','Pred','Fact'],1,inplace=True)


In [136]:
pivot_df = df.pivot(index='Date', columns='Asset', values='pct_pred')


# Заменим названия колонок для удобства
pivot_df.columns.name = None

In [138]:
pivot_df.to_csv('cvxportfolio_pred_returns')